In [4]:
ls APAC_2023_Datasets/Crashes

crash_info_commericial_vehicles.csv  crash_info_people.csv
crash_info_flag_variables.csv        crash_info_roadway.csv
crash_info_general.csv               crash_info_trailed_vehicles.csv
crash_info_motorcycle.csv            crash_info_vehicles.csv


In [5]:
ls "APAC_2023_Datasets/Traffic, Investigations _ Other"

crimes.csv                           police_districts.csv
hourly_weather_philadelphia.csv      police_stations.csv
investigations.csv                   traffic_stops_philadelphia.csv
philadelphia_population_metrics.csv


In [6]:
ls "Data Open 2023 Additional Data"

Boundaries_District.geojson  police_districts.geojson
philadelphia.geojson         Vital_Population_CT.geojson


In [8]:
import pandas as pd
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
import numpy as np

Our goal is to link all the data together using the police district number. 

Each row will represent the features of that police district (eg. car crash, crime) in a weekly/monthly basis

## Load related data

In [11]:
with open('Data Open 2023 Additional Data/philadelphia.geojson') as handle:
    philly_geo = json.loads(handle.read())
  
district_gdf = gpd.read_file('Data Open 2023 Additional Data/Boundaries_District.geojson')

## Police District

In [9]:
df_police_districts = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/police_districts.csv")
df_police_districts.head()

,OBJECTID,PERIMETER,DISTRICT_NUM,LOCATION,LAT,LNG,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,343,69282.58846,16,39th St. & Lancaster Ave.,39.96158,-75.19940,SWPD,1.216700e+08,1.927226e+07,27575.07918
1,344,33150.15496,17,20th St. & Federal St.,39.93728,-75.17670,SPD,5.786368e+07,9.154950e+06,13179.95335
2,345,54403.93004,18,55th St. & Pine St.,39.95426,-75.23217,SWPD,9.881929e+07,1.564117e+07,21629.61069
3,346,51597.05160,35,N. Broad St. & Champlost St.,40.04442,-75.14351,NWPD,1.544309e+08,2.450685e+07,20554.86914
4,347,58075.01444,39,22nd St. & Hunting Park Ave.,40.01362,-75.15884,NWPD,1.578976e+08,2.503787e+07,23123.58026


## Crime with district

In [12]:
crime = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/crimes.csv")

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(crime['lng'], crime['lat'])]
crime_gdf = gpd.GeoDataFrame(crime, crs="EPSG:4326", geometry=geometry)
crime_with_district = gpd.sjoin(crime_gdf, district_gdf, how='left', op='within')
crime_with_district = crime_with_district.dropna(subset=["DIST_NUM"])
print(len(crime_with_district))
crime_with_district.head()

/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


1573848


,objectid,psa,dispatch_date,dispatch_time,location_block,text_general_code,lat,lng,fips,geometry,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
0,218891,2,2022-09-08,15:21:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,NaN,39.0,NaN,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
1,218892,2,2022-09-14,17:41:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,NaN,39.0,NaN,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
2,218893,2,2022-09-19,15:23:00,3200 BLOCK N BROAD ST,Thefts,40.002741,-75.153068,4.210102e+11,POINT (-75.15307 40.00274),...,NaN,39.0,NaN,39,22nd St. & Hunting Park Ave.,686-3390,NWPD,1.578976e+08,2.503787e+07,23123.580262
3,218894,2,2022-08-22,09:38:00,0 BLOCK WOLF ST,Theft from Vehicle,39.918351,-75.145999,4.210198e+11,POINT (-75.14600 39.91835),...,NaN,3.0,NaN,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793
4,218895,2,2022-08-20,10:31:00,0 BLOCK WOLF ST,Theft from Vehicle,39.918351,-75.145999,4.210198e+11,POINT (-75.14600 39.91835),...,NaN,3.0,NaN,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


In [13]:
crime_with_district.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1573848 entries, 0 to 1593141
Data columns (total 25 columns):
 #   Column             Non-Null Count    Dtype   
---  ------             --------------    -----   
 0   objectid           1573848 non-null  int64   
 1   psa                1573247 non-null  object  
 2   dispatch_date      1573848 non-null  object  
 3   dispatch_time      1573848 non-null  object  
 4   location_block     1573791 non-null  object  
 5   text_general_code  1573848 non-null  object  
 6   lat                1573848 non-null  float64 
 7   lng                1573848 non-null  float64 
 8   fips               1573848 non-null  float64 
 9   geometry           1573848 non-null  geometry
 10  index_right        1573848 non-null  float64 
 11  OBJECTID           1573848 non-null  float64 
 12  AREA               0 non-null        float64 
 13  PERIMETER          1573848 non-null  float64 
 14  DISTRICT_          1573848 non-null  float64 
 15  DISTRIC

In [50]:
# We will keep the date and the types of crime (text_general_code)
crime_useful_feat = crime_with_district[["dispatch_date", "text_general_code", "DIST_NUM"]]
crime_useful_feat = crime_useful_feat.join(pd.get_dummies(crime_with_district["text_general_code"]))
crime_useful_feat['dispatch_date'] = pd.to_datetime(crime_useful_feat['dispatch_date'])
crime_useful_feat['weekday'] = crime_useful_feat['dispatch_date'].dt.weekday+1    # 1=Mon, 2=Tue, 3=Wed, ... ,8=Unknown
crime_useful_feat_weekday = crime_useful_feat.groupby(["DIST_NUM", pd.Grouper(key='dispatch_date', freq='1M', origin="epoch"), "weekday"]).sum()   # no. of crimes per week/month
crime_useful_feat_weekday['crime_total'] = crime_useful_feat_weekday.sum(axis=1)
crime_useful_feat_month = crime_useful_feat.groupby(["DIST_NUM", pd.Grouper(key='dispatch_date', freq='1M', origin="epoch")]).sum()   # no. of crimes per week/month
crime_useful_feat_month = crime_useful_feat_month.drop("weekday", axis=1)
crime_useful_feat_month['crime_total'] = crime_useful_feat_month.sum(axis=1)
crime_useful_feat_weekday.head()

<ipython-input-50-4607d6d4023d>:6: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  crime_useful_feat_weekday = crime_useful_feat.groupby(["DIST_NUM", pd.Grouper(key='dispatch_date', freq='1M', origin="epoch"), "weekday"]).sum()   # no. of crimes per week/month
<ipython-input-50-4607d6d4023d>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  crime_useful_feat_month = crime_useful_feat.groupby(["DIST_NUM", pd.Grouper(key='dispatch_date', freq='1M', origin="epoch")]).sum()   # no. of crimes per week/month


Aggravated Assault Firearm  \
DIST_NUM dispatch_date weekday                               
1.0      2012-12-31    1                                 0   
         2013-01-31    1                                 0   
                       2                                 0   
                       3                                 0   
                       4                                 0   

                                Aggravated Assault No Firearm  \
DIST_NUM dispatch_date weekday                                  
1.0      2012-12-31    1                                    0   
         2013-01-31    1                                    0   
                       2                                    0   
                       3                                    2   
                       4                                    0   

                                All Other Offenses  Arson  \
DIST_NUM dispatch_date weekday                              
1.0      2012-12-31    1                         0      0   
         2013-01-31    1                         6      0   
                       2                         2      0   
                       3                         3      0   
                       4                         6      0   

                                Burglary Non-Residential  \
DIST_NUM dispatch_date weekday                             
1.0      2012-12-31    1                               0   
         2013-01-31    1                               3   
                       2                               0   
                       3                               2   
                       4                               0   

                                Burglary Residential  \
DIST_NUM dispatch_date weekday                         
1.0      2012-12-31    1                           0   
         2013-01-31    1                           1   
                       2                           0   
                       3                           3   
                       4                           3   

                                DRIVING UNDER THE INFLUENCE  \
DIST_NUM dispatch_date weekday                                
1.0      2012-12-31    1                                  0   
         2013-01-31    1                                  2   
                       2                                  1   
                       3                                  0   
                       4                                  2   

                                Disorderly Conduct  Embezzlement  \
DIST_NUM dispatch_date weekday                                     
1.0      2012-12-31    1                         0             0   
         2013-01-31    1                         0             0   
                       2                         0             0   
                       3                         0             1   
                       4                         0             0   

                                Forgery and Counterfeiting  ...  Rape  \
DIST_NUM dispatch_date weekday                              ...         
1.0      2012-12-31    1                                 0  ...     0   
         2013-01-31    1                                 0  ...     0   
                       2                                 0  ...     0   
                       3                                 1  ...     1   
                       4                                 0  ...     0   

                                Receiving Stolen Property  Robbery Firearm  \
DIST_NUM dispatch_date weekday                                               
1.0      2012-12-31    1                                0                0   
         2013-01-31    1                                0                0   
                       2                                0                0   
                       3                                0                0   


## Crash with District

In [38]:
df_crash = pd.read_csv("APAC_2023_Datasets/Crashes/crash_info_general.csv")

# create Geom column in crime_df using lat and lon
geometry = [Point(xy) for xy in zip(df_crash['DEC_LONG'], df_crash['DEC_LAT'])]
crash_gdf = gpd.GeoDataFrame(df_crash, crs="EPSG:4326", geometry=geometry)
crash_with_district = gpd.sjoin(crash_gdf, district_gdf, how='left', op='within')
crash_with_district = crash_with_district.dropna(subset=["DIST_NUM"])
print(len(crash_with_district))
crash_with_district.head()

<ipython-input-38-a4db084c09cf>:1: DtypeWarning: Columns (97,98,99) have mixed types. Specify dtype option on import or set low_memory=False.
  df_crash = pd.read_csv("APAC_2023_Datasets/Crashes/crash_info_general.csv")
/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py:3473: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


131569


,Unnamed: 0,CRN,ARRIVAL_TM,AUTOMOBILE_COUNT,BELTED_DEATH_COUNT,BELTED_SUSP_SERIOUS_INJ_COUNT,BICYCLE_COUNT,BICYCLE_DEATH_COUNT,BICYCLE_SUSP_SERIOUS_INJ_COUNT,BUS_COUNT,...,DISTRICT_ID,DIST_NUM,SUM_AREA,DIST_NUMC,LOCATION,PHONE,DIV_CODE,AREA_SQMI,Shape__Area,Shape__Length
1,2,2010004457,658.0,2,0,0,0,0,0,0,...,NaN,6.0,NaN,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
2,3,2010005005,1217.0,2,0,0,0,0,0,0,...,NaN,6.0,NaN,06,11th St. & Winter St.,686-3060,CPD,6.927927e+07,1.096654e+07,13788.505692
4,5,2010005373,36.0,0,0,0,0,0,0,0,...,NaN,16.0,NaN,16,39th St. & Lancaster Ave.,686-3160,SWPD,1.216700e+08,1.927226e+07,27575.079183
5,6,2010005374,326.0,1,0,0,0,0,0,0,...,NaN,15.0,NaN,15,Harbison Ave. & Levick St.,686-3150,NEPD,3.077864e+08,4.881580e+07,35944.018281
6,7,2010005375,505.0,1,0,0,0,0,0,0,...,NaN,3.0,NaN,03,11th St. & Wharton St.,686-3030,SPD,1.839049e+08,2.907948e+07,22002.676793


In [ ]:
# We will keep the following features
# categorical feat: *_TYPE, ILLUMINATION, MAX_SEVERITY_LEVEL, RELATION_TO_ROAD, ROAD_CONDITION, URBAN_RURAL, WEATHER1
# numerical feat: *_COUNT
# date feat: CRASH_MONTH, CRASH_YEAR, DATE_OF_WEEK

In [83]:
manual_cat_list = ["ILLUMINATION", "MAX_SEVERITY_LEVEL", "RELATION_TO_ROAD", "ROAD_CONDITION", "URBAN_RURAL", "WEATHER1"]
manual_drop_list = ["RDWY_SURF_TYPE_CD", "WORK_ZONE_TYPE"]
crash_useful_cat_feat_list = [i for i in crash_with_district.columns if ("_TYPE" in i and i not in manual_drop_list) or i in manual_cat_list]
crash_useful_num_feat_list = [i for i in crash_with_district.columns if "_COUNT" in i]

crash_useful_cat_feat = pd.get_dummies(crash_with_district[crash_useful_cat_feat_list].astype(str))
crash_useful_feat = pd.concat([crash_with_district[crash_useful_num_feat_list], crash_useful_cat_feat], axis=1)
crash_useful_feat['date'] = pd.to_datetime(crash_with_district['CRASH_YEAR'].astype(str)+"-"+crash_with_district['CRASH_MONTH'].astype(str)+"-1")
crash_useful_feat['weekday'] = (crash_with_district['DAY_OF_WEEK']-1).replace(0, 7)   # fit with the original weekday code
crash_useful_feat['DIST_NUM'] = crash_with_district['DIST_NUM']

In [84]:
crash_useful_feat_weekday = crash_useful_feat.groupby(["DIST_NUM", pd.Grouper(key='date', freq='1M', origin="epoch"), "weekday"]).sum()   # no. of crash per week/month
crash_useful_feat_weekday['crash_total'] = crash_useful_feat_weekday.sum(axis=1)
crash_useful_feat_month = crash_useful_feat.groupby(["DIST_NUM", pd.Grouper(key='date', freq='1M', origin="epoch")]).sum()   # no. of crash per week/month
crash_useful_feat_month = crash_useful_feat_month.drop("weekday", axis=1)
crash_useful_feat_month['crash_total'] = crash_useful_feat_month.sum(axis=1)
crash_useful_feat_weekday.head()

AUTOMOBILE_COUNT  BELTED_DEATH_COUNT  \
DIST_NUM date       weekday                                         
1.0      2010-01-31 3                       2                   0   
                    4                       7                   0   
                    5                       2                   0   
                    6                       5                   0   
                    7                       8                   0   

                             BELTED_SUSP_SERIOUS_INJ_COUNT  BICYCLE_COUNT  \
DIST_NUM date       weekday                                                 
1.0      2010-01-31 3                                    0              0   
                    4                                    0              0   
                    5                                    0              0   
                    6                                    0              0   
                    7                                    0              0   

                             BICYCLE_DEATH_COUNT  \
DIST_NUM date       weekday                        
1.0      2010-01-31 3                          0   
                    4                          0   
                    5                          0   
                    6                          0   
                    7                          0   

                             BICYCLE_SUSP_SERIOUS_INJ_COUNT  BUS_COUNT  \
DIST_NUM date       weekday                                              
1.0      2010-01-31 3                                     0          0   
                    4                                     0          1   
                    5                                     0          0   
                    6                                     0          0   
                    7                                     0          0   

                             CHLDPAS_DEATH_COUNT  \
DIST_NUM date       weekday                        
1.0      2010-01-31 3                          0   
                    4                          0   
                    5                          0   
                    6                          0   
                    7                          0   

                             CHLDPAS_SUSP_SERIOUS_INJ_COUNT  COMM_VEH_COUNT  \
DIST_NUM date       weekday                                                   
1.0      2010-01-31 3                                     0               0   
                    4                                     0               1   
                    5                                     0               0   
                    6                                     0               0   
                    7                                     0               0   

                             ...  WEATHER1_3  WEATHER1_4  WEATHER1_5  \
DIST_NUM date       weekday  ...                                       
1.0      2010-01-31 3        ...           1           0           0   
                    4        ...           5           0           0   
                    5        ...           1           0           0   
                    6        ...           3           0           0   
                    7        ...           4           0           0   

                             WEATHER1_6  WEATHER1_7  WEATHER1_8  WEATHER1_9  \
DIST_NUM date       weekday                                                   
1.0      2010-01-31 3                 0           0           0           0   
                    4                 0           0           0           0   
                    5                 0           0           0           0   
                    6                 0           0           0           0   
                    7                 0           2           0           0   

                             WEATHER1_98  WEATHER1_99  crash_total  
DIST_NUM date       weekday                                         
1.0  

## Traffic Stop Data

In [115]:
traffic_stop_with_district = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/traffic_stops_philadelphia.csv")
traffic_stop_with_district.head()

<ipython-input-115-9a81bfa2389d>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  traffic_stop_with_district = pd.read_csv("APAC_2023_Datasets/Traffic, Investigations _ Other/traffic_stops_philadelphia.csv")


,objectid,date,time,location,lat,lng,district,service_area,subject_age,subject_race,...,outcome,contraband_found,frisk_performed,search_conducted,search_person,search_vehicle,raw_race,raw_individual_contraband,raw_vehicle_contraband,fips
0,1,2014-01-01,01:14:00,NaN,NaN,NaN,19.0,191,31.0,black,...,arrest,True,False,True,True,False,Black - Non-Latino,True,False,NaN
1,2,2014-01-01,01:57:00,NaN,NaN,NaN,12.0,121,21.0,black,...,arrest,False,True,True,True,False,Black - Non-Latino,False,False,NaN
2,3,2014-01-01,03:30:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,24.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
3,4,2014-01-01,03:40:00,3400 BLOCK SPRUCE ST,39.950424,-75.192680,18.0,183,20.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210104e+11
4,5,2014-01-01,08:30:00,N 56TH ST / UPLAND WAY,39.983712,-75.234188,19.0,193,31.0,black,...,NaN,NaN,False,False,False,False,Black - Non-Latino,False,False,4.210101e+11


In [119]:
# cat feat: subject race, subject sex, type, arrest_made, outcome, contraband_found, frisk_performed, search_conducted

traffic_stop_useful_cat_feat = traffic_stop_with_district[["subject_race", "subject_sex", "type", "arrest_made", "outcome", "contraband_found", "frisk_performed", "search_conducted"]]
traffic_stop_useful_cat_feat["outcome"] = traffic_stop_useful_cat_feat['outcome'].fillna("none")
traffic_stop_useful_cat_feat["contraband_found"] = traffic_stop_useful_cat_feat['contraband_found'].fillna(False).astype(bool)
traffic_stop_useful_cat_feat.info()

<ipython-input-119-212e7b358728>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traffic_stop_useful_cat_feat["outcome"] = traffic_stop_useful_cat_feat['outcome'].fillna("none")
<ipython-input-119-212e7b358728>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traffic_stop_useful_cat_feat["contraband_found"] = traffic_stop_useful_cat_feat['contraband_found'].fillna(False).astype(bool)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1865096 entries, 0 to 1865095
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   subject_race      object
 1   subject_sex       object
 2   type              object
 3   arrest_made       bool  
 4   outcome           object
 5   contraband_found  bool  
 6   frisk_performed   bool  
 7   search_conducted  bool  
dtypes: bool(4), object(4)
memory usage: 64.0+ MB


In [120]:
traffic_stop_useful_feat = pd.concat([pd.get_dummies(traffic_stop_useful_cat_feat[["subject_race", "subject_sex", "type", "outcome"]]), 
                                      traffic_stop_useful_cat_feat[["contraband_found", "frisk_performed", "search_conducted","arrest_made"]]], axis=1)
traffic_stop_useful_feat['date'] = pd.to_datetime(traffic_stop_with_district['date'])
traffic_stop_useful_feat['district'] = traffic_stop_with_district['district']
traffic_stop_useful_feat['weekday'] = traffic_stop_useful_feat['date'].dt.weekday+1    # 1=Mon, 2=Tue, 3=Wed, ... ,8=Unknown
traffic_stop_useful_feat_weekday = traffic_stop_useful_feat.groupby(["district", pd.Grouper(key='date', freq='1M', origin="epoch"), "weekday"]).sum()   # no. of traffic_stops per week/month
traffic_stop_useful_feat_weekday['traffic_stop_total'] = traffic_stop_useful_feat_weekday.sum(axis=1)
traffic_stop_useful_feat_month = traffic_stop_useful_feat.groupby(["district", pd.Grouper(key='date', freq='1M', origin="epoch")]).sum()   # no. of traffic_stops per week/month
traffic_stop_useful_feat_month = traffic_stop_useful_feat_month.drop("weekday", axis=1)
traffic_stop_useful_feat_month['traffic_stop_total'] = traffic_stop_useful_feat_month.sum(axis=1)
traffic_stop_useful_feat_weekday.head()

subject_race_asian/pacific islander  \
district date       weekday                                        
1.0      2014-01-31 1                                          4   
                    2                                          2   
                    3                                          6   
                    4                                          3   
                    5                                          6   

                             subject_race_black  subject_race_hispanic  \
district date       weekday                                              
1.0      2014-01-31 1                        31                      0   
                    2                        31                      3   
                    3                        27                      0   
                    4                        22                      1   
                    5                        29                      1   

                             subject_race_other  subject_race_unknown  \
district date       weekday                                             
1.0      2014-01-31 1                         0                     0   
                    2                         1                     0   
                    3                         1                     1   
                    4                         0                     1   
                    5                         0                     2   

                             subject_race_white  subject_sex_female  \
district date       weekday                                           
1.0      2014-01-31 1                        19                  14   
                    2                        19                  19   
                    3                        32                  12   
                    4                        30                  11   
                    5                        31                  16   

                             subject_sex_male  type_pedestrian  \
district date       weekday                                      
1.0      2014-01-31 1                      40               28   
                    2                      37               21   
                    3                      55               29   
                    4                      46               19   
                    5                      53               24   

                             type_vehicular  outcome_arrest  outcome_none  \
district date       weekday                                                 
1.0      2014-01-31 1                    26               1            53   
                    2                    35               2            54   
                    3                    38               3            64   
                    4                    38               2            55   
                    5                    45               2            67   

                             contraband_found  frisk_performed  \
district date       weekday                                      
1.0      2014-01-31 1                       1                4   
                    2                       0                3   
                    3                       1                7   
                    4                       0                1   
                    5                       0                4   

                             search_conducted  arrest_made  traffic_stop_total  
district date       weekday                                                     
1.0      2014-01-31 1                       1            1               223.0  
                    2                       2            2               231.0  
                    3                       3            3               282.0  
                    4                       2            2               233.0  
                    5                       3            2             

## Merge Crime, Crash, Traffic Stop

In [125]:
crime_useful_feat_month.columns = ["crime_"+i if "crime_total" not in i else i for i in crime_useful_feat_month.columns ]
crash_useful_feat_month.columns = ["crash_"+i if "crash_total" not in i else i for i in crash_useful_feat_month.columns]
traffic_stop_useful_feat_month.columns = ["traffic_stop_"+i if "traffic_stop_total" not in i else i for i in traffic_stop_useful_feat_month.columns]

In [137]:
#from December 2013 - April 2018.
df_monthly = pd.concat([pd.concat([crime_useful_feat_month, crash_useful_feat_month], axis=1), traffic_stop_useful_feat_month], axis=1)
df_monthly = df_monthly.dropna(axis=0)
df_monthly.head()

crime_crime_Aggravated Assault Firearm  \
1.0 2014-01-31                                     1.0   
    2014-02-28                                     2.0   
    2014-03-31                                     3.0   
    2014-04-30                                     5.0   
    2014-05-31                                     1.0   

                crime_crime_Aggravated Assault No Firearm  \
1.0 2014-01-31                                        9.0   
    2014-02-28                                        8.0   
    2014-03-31                                       10.0   
    2014-04-30                                        8.0   
    2014-05-31                                       12.0   

                crime_crime_All Other Offenses  crime_crime_Arson  \
1.0 2014-01-31                            26.0                0.0   
    2014-02-28                            46.0                0.0   
    2014-03-31                            83.0                1.0   
    2014-04-30                            78.0                1.0   
    2014-05-31                            55.0                0.0   

                crime_crime_Burglary Non-Residential  \
1.0 2014-01-31                                   1.0   
    2014-02-28                                   1.0   
    2014-03-31                                   1.0   
    2014-04-30                                   3.0   
    2014-05-31                                   1.0   

                crime_crime_Burglary Residential  \
1.0 2014-01-31                              10.0   
    2014-02-28                               7.0   
    2014-03-31                               6.0   
    2014-04-30                              10.0   
    2014-05-31                              11.0   

                crime_crime_DRIVING UNDER THE INFLUENCE  \
1.0 2014-01-31                                      7.0   
    2014-02-28                                      8.0   
    2014-03-31                                      6.0   
    2014-04-30                                      7.0   
    2014-05-31                                      4.0   

                crime_crime_Disorderly Conduct  crime_crime_Embezzlement  \
1.0 2014-01-31                             3.0                       0.0   
    2014-02-28                             2.0                       0.0   
    2014-03-31                             5.0                       1.0   
    2014-04-30                             3.0                       0.0   
    2014-05-31                             6.0                       2.0   

                crime_crime_Forgery and Counterfeiting  ...  \
1.0 2014-01-31                                     2.0  ...   
    2014-02-28                                     1.0  ...   
    2014-03-31                                     0.0  ...   
    2014-04-30                                     1.0  ...   
    2014-05-31                                     1.0  ...   

                traffic_stop_subject_sex_male  traffic_stop_type_pedestrian  \
1.0 2014-01-31                          286.0                         146.0   
    2014-02-28                          857.0                         519.0   
    2014-03-31                          760.0                         481.0   
    2014-04-30                         1108.0                         730.0   
    2014-05-31                         1054.0                         690.0   

                traffic_stop_type_vehicular  traffic_stop_outcome_arrest  \
1.0 2014-01-31                        236.0                         12.0   
    2014-02-28                        613.0                         39.0   
    2014-03-31                        512.0                         28.0   
    2014-04-30                        755.0                         52.0   
    2014-05-31                        729.0                         45.0   

                traffic_stop_outcome_none  traffic_stop_contraband_found  \
1.0 2014-01-31                      

In [138]:
crime_useful_feat_weekday.columns = ["crime_"+i if "crime_total" not in i else i for i in crime_useful_feat_weekday.columns ]
crash_useful_feat_weekday.columns = ["crash_"+i if "crash_total" not in i else i for i in crash_useful_feat_weekday.columns]
traffic_stop_useful_feat_weekday.columns = ["traffic_stop_"+i if "traffic_stop_total" not in i else i for i in traffic_stop_useful_feat_weekday.columns]

In [139]:
#from December 2013 - April 2018.
df_weekday = pd.concat([pd.concat([crime_useful_feat_weekday, crash_useful_feat_weekday], axis=1), traffic_stop_useful_feat_weekday], axis=1)
df_weekday = df_weekday.dropna(axis=0)
df_weekday.head()

crime_Aggravated Assault Firearm  \
               weekday                                     
1.0 2014-01-31 1                                     0.0   
               2                                     0.0   
               3                                     0.0   
               4                                     1.0   
               5                                     0.0   

                        crime_Aggravated Assault No Firearm  \
               weekday                                        
1.0 2014-01-31 1                                        1.0   
               2                                        0.0   
               3                                        1.0   
               4                                        1.0   
               5                                        4.0   

                        crime_All Other Offenses  crime_Arson  \
               weekday                                          
1.0 2014-01-31 1                             6.0          0.0   
               2                             4.0          0.0   
               3                             3.0          0.0   
               4                             4.0          0.0   
               5                             3.0          0.0   

                        crime_Burglary Non-Residential  \
               weekday                                   
1.0 2014-01-31 1                                   1.0   
               2                                   0.0   
               3                                   0.0   
               4                                   0.0   
               5                                   0.0   

                        crime_Burglary Residential  \
               weekday                               
1.0 2014-01-31 1                               2.0   
               2                               2.0   
               3                               0.0   
               4                               0.0   
               5                               2.0   

                        crime_DRIVING UNDER THE INFLUENCE  \
               weekday                                      
1.0 2014-01-31 1                                      2.0   
               2                                      0.0   
               3                                      0.0   
               4                                      2.0   
               5                                      3.0   

                        crime_Disorderly Conduct  crime_Embezzlement  \
               weekday                                                 
1.0 2014-01-31 1                             0.0                 0.0   
               2                             1.0                 0.0   
               3                             0.0                 0.0   
               4                             1.0                 0.0   
               5                             1.0                 0.0   

                        crime_Forgery and Counterfeiting  ...  \
               weekday                                    ...   
1.0 2014-01-31 1                                     0.0  ...   
               2                                     0.0  ...   
               3                                     0.0  ...   
               4                                     0.0  ...   
               5                                     0.0  ...   

                        traffic_stop_subject_sex_male  \
               weekday                                  
1.0 2014-01-31 1                                 40.0   
               2                                 37.0   
               3                                 55.0   
               4                                 46.0   
               5                                 53.0   

                        traffic_stop_type_pedestrian  \
               weekday                                 
1.0 2014-01-31 1  

In [141]:
df_monthly.to_csv("monthly_district_data.csv")
df_weekday.to_csv("weekday_district_data.csv")